<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Importar os módulos

In [1]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import Dataset_transf as dprep
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import metrics, model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, roc_curve, precision_score, recall_score, confusion_matrix, f1_score, fbeta_score, make_scorer
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import model_selection

# Obter os dataframes dos ficheiros

In [2]:
metrics_dict={}
tentativa = 0

In [3]:
failures_df, signals_df, logs_df = dprep.get_data()

In [4]:
failures_df[failures_df.Component == 'HYDRAULIC_GROUP']

,Turbine_ID,Component,Timestamp
1,T06,HYDRAULIC_GROUP,2016-04-04 18:50:00
14,T11,HYDRAULIC_GROUP,2016-10-17 17:40:00
17,T11,HYDRAULIC_GROUP,2017-04-26 18:00:00
18,T07,HYDRAULIC_GROUP,2017-06-17 11:30:00
20,T06,HYDRAULIC_GROUP,2017-08-19 09:40:00


In [5]:
failures_df.Component.value_counts()

GENERATOR            7
GENERATOR_BEARING    6
HYDRAULIC_GROUP      5
TRANSFORMER          3
GEARBOX              2
Name: Component, dtype: int64

In [6]:
df_dict = {'failures_df':failures_df, 'signals_df':signals_df, 'logs_df':logs_df}

In [7]:
for key in df_dict:
    print(key, df_dict[key].shape)

failures_df (23, 3)
signals_df (434145, 81)
logs_df (256231, 3)


# Criação de datasets de components

In [8]:
df_generator, df_gen_bear, df_transformer, df_hydraulic, df_gearbox = dprep.component_df_creation(signals_df)

In [9]:
comp_df_dict = {
    'df_generator': df_generator,
    'df_hydraulic': df_hydraulic,
    'df_gen_bear': df_gen_bear,
    'df_transformer': df_transformer,
    'df_gearbox': df_gearbox
}

In [10]:
for key in comp_df_dict:
    print(comp_df_dict[key].shape, key, )

(434145, 45) df_generator
(434145, 34) df_hydraulic
(434145, 45) df_gen_bear
(434145, 36) df_transformer
(434145, 33) df_gearbox


In [11]:
component_list = failures_df.Component.unique()
component_list

array(['GENERATOR', 'HYDRAULIC_GROUP', 'GENERATOR_BEARING', 'TRANSFORMER',
       'GEARBOX'], dtype=object)

# Cópia dos df para só carregar uma vez no ficheiro

In [76]:
comp_prep_df_dict = comp_df_dict.copy()

In [77]:
for key, i in enumerate(comp_prep_df_dict):
    print(key, i)

0 df_generator
1 df_hydraulic
2 df_gen_bear
3 df_transformer
4 df_gearbox


# Merge dos DF com o DF de failures

In [78]:
for i, key in enumerate(comp_prep_df_dict):
    comp_prep_df_dict[key] = dprep.sig_fail_merge_dfs(sig_df=comp_prep_df_dict[key],
                                               fail_df=failures_df,
                                               component=component_list[i])

KeyboardInterrupt: 

In [ ]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

# Fillna by turbine

In [ ]:
turbine_list = signals_df.Turbine_ID.unique()
turbine_list

In [ ]:
for i, key in enumerate(comp_prep_df_dict):
    comp_prep_df_dict[key] = dprep.fill_na_by_turbine(comp_prep_df_dict[key],
                                                      turbine_list)

In [ ]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

# Criação de Variável alvo

In [ ]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].isna().sum().value_counts(), key)

In [ ]:
days_list = [60]

In [ ]:
for day in days_list:
    for key in comp_prep_df_dict:
        comp_prep_df_dict[key] = dprep.aplic_var_target(comp_prep_df_dict[key], day)

In [ ]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

# Feature selection

In [ ]:
gen_features_drop = [
    'Gen_RPM_Max', 'Gen_RPM_Min', 'Gen_Phase1_Temp_Avg', 'Gen_Phase3_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Rtr_RPM_Max', 'Rtr_RPM_Min',
    'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Prod_LatestAvg_ReactPwrGen1', 'Cont_Hub_Temp_Avg', 'Spin_Temp_Avg',
    'Rtr_RPM_Std', 'Rtr_RPM_Avg', 'Cont_VCP_Temp_Avg'
]

In [ ]:
gen_bear_features_drop = [
    'Gen_RPM_Max', 'Gen_RPM_Min', 'Gen_Phase1_Temp_Avg', 'Gen_Phase3_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Rtr_RPM_Max', 'Rtr_RPM_Min',
    'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Prod_LatestAvg_ReactPwrGen1', 'Cont_Hub_Temp_Avg', 'Spin_Temp_Avg',
    'Rtr_RPM_Std', 'Rtr_RPM_Avg', 'Cont_VCP_Temp_Avg'
]

In [ ]:
hyd_features_drop = [
    'Rtr_RPM_Max', 'Rtr_RPM_Min', 'Blds_PitchAngle_Max', 'Blds_PitchAngle_Min',
    'Blds_PitchAngle_Max', 'Grd_RtrInvPhase3_Temp_Avg', 'Grd_Busbar_Temp_Avg',
    'Amb_WindSpeed_Est_Avg', 'Spin_Temp_Avg', 'Cont_Hub_Temp_Avg',
    'Grd_RtrInvPhase1_Temp_Avg', 'Cont_VCP_Temp_Avg'
]

In [ ]:
gearbox_features_drop = [
    'Rtr_RPM_Max', 'Rtr_RPM_Min', 'Grd_RtrInvPhase1_Temp_Avg',
    'Grd_RtrInvPhase3_Temp_Avg', 'Blds_PitchAngle_Min', 'Blds_PitchAngle_Max',
    'Cont_VCP_Temp_Avg', 'Grd_Busbar_Temp_Avg', 'Amb_WindSpeed_Est_Avg',
    'Spin_Temp_Avg'
]

In [ ]:
transf_features_drop = [
    'HVTrafo_Phase1_Temp_Avg', 'HVTrafo_Phase3_Temp_Avg', 'Rtr_RPM_Max',
    'Rtr_RPM_Min', 'Grd_RtrInvPhase1_Temp_Avg', 'Grd_RtrInvPhase3_Temp_Avg',
    'Blds_PitchAngle_Min', 'Blds_PitchAngle_Max', 'Amb_WindSpeed_Est_Avg',
    'Spin_Temp_Avg', 'Cont_VCP_Temp_Avg'
]

In [ ]:
feat_drop_list = [
    gen_features_drop, hyd_features_drop, gen_bear_features_drop,
    transf_features_drop, gearbox_features_drop
]

In [ ]:
for i, key in enumerate(comp_prep_df_dict):
    print(f'A actualizar dataframe {i+1} de {len(comp_prep_df_dict)}')
#     for col in feat_drop_list:
#         print(f'A apagar a coluna {col}')
    comp_prep_df_dict[key] = comp_prep_df_dict[key].drop(columns=feat_drop_list[i])

In [ ]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

# Agrupar pela medida de tempo seleccionada.

In [ ]:
# Passagem para dias
Agrup_por = 'Dia'
for key in comp_prep_df_dict:
    comp_prep_df_dict[key] = dprep.group_por_frequency(comp_prep_df_dict[key], period=Agrup_por)

In [ ]:
# Passagem para dias 
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

# Adicionar as medidas de alisamento

In [ ]:
for key in comp_prep_df_dict:
    comp_prep_df_dict[key] = dprep.add_features(comp_prep_df_dict[key], rolling_win_size=10)

In [ ]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape, key)

In [ ]:
# # Para considerar apenas as rubricas de alisamento
# cols_to_drop_generator = [
#     'Amb_Temp_Avg', 'Amb_WindDir_Abs_Avg', 'Amb_WindDir_Relative_Avg',
#     'Amb_WindSpeed_Avg', 'Amb_WindSpeed_Max', 'Amb_WindSpeed_Min',
#     'Amb_WindSpeed_Std', 'Blds_PitchAngle_Avg', 'Blds_PitchAngle_Std',
#     'Cont_Top_Temp_Avg', 'Cont_VCP_ChokcoilTemp_Avg',
#     'Cont_VCP_WtrTemp_Avg', 'Gen_Bear2_Temp_Avg', 'Gen_Bear_Temp_Avg',
#     'Gen_Phase2_Temp_Avg', 'Gen_RPM_Avg', 'Gen_RPM_Std',
#     'Gen_SlipRing_Temp_Avg', 'Grd_Busbar_Temp_Avg',
#     'Grd_RtrInvPhase2_Temp_Avg', 'Hyd_Oil_Temp_Avg', 'Nac_Direction_Avg',
#     'Nac_Temp_Avg', 'Prod_LatestAvg_ActPwrGen0', 'Prod_LatestAvg_ActPwrGen1',
#     'Prod_LatestAvg_ReactPwrGen0'
# ]

In [ ]:
# # Para considerar apenas as rubricas de alisamento
# cols_to_drop_hyd_group = [
#     'Amb_WindDir_Abs_Avg', 'Amb_WindDir_Relative_Avg',
#     'Amb_WindSpeed_Avg', 'Amb_WindSpeed_Max', 'Amb_WindSpeed_Min',
#     'Amb_WindSpeed_Std', 'Blds_PitchAngle_Avg', 'Blds_PitchAngle_Std',
#     'Nac_Direction_Avg', 'Rtr_RPM_Avg', 'Rtr_RPM_Std'
# ]

In [ ]:
# # Para considerar apenas as rubricas de alisamento
# cols_to_drop_gen_bear =  ['Amb_WindDir_Abs_Avg', 'Amb_WindDir_Relative_Avg',
#        'Amb_WindSpeed_Avg', 'Amb_WindSpeed_Max', 'Amb_WindSpeed_Min',
#        'Amb_WindSpeed_Std', 'Blds_PitchAngle_Avg', 'Blds_PitchAngle_Std',
#        'Gen_Bear_Temp_Avg', 'Gen_RPM_Avg', 'Gen_RPM_Std',
#        'Nac_Direction_Avg', 'Prod_LatestAvg_ActPwrGen1']

In [ ]:
# # Para considerar apenas as rubricas de alisamento
# cols_to_drop_transf = ['Amb_WindDir_Abs_Avg', 'Amb_WindDir_Relative_Avg',
#        'Amb_WindSpeed_Avg', 'Amb_WindSpeed_Max', 'Amb_WindSpeed_Min',
#        'Amb_WindSpeed_Std', 'Blds_PitchAngle_Avg', 'Blds_PitchAngle_Std',
#        'Grd_Prod_VoltPhse1_Avg', 'Grd_Prod_VoltPhse2_Avg',
#        'Grd_Prod_VoltPhse3_Avg', 'Nac_Direction_Avg', 'Rtr_RPM_Avg',
#        'Rtr_RPM_Std']

In [ ]:
# # Para considerar apenas as rubricas de alisamento
# cols_to_drop_gearbox = ['Amb_WindDir_Abs_Avg', 'Amb_WindDir_Relative_Avg',
#        'Amb_WindSpeed_Avg', 'Amb_WindSpeed_Max', 'Amb_WindSpeed_Min',
#        'Amb_WindSpeed_Std', 'Blds_PitchAngle_Avg', 'Blds_PitchAngle_Std',
#        'Nac_Direction_Avg', 'Rtr_RPM_Avg', 'Rtr_RPM_Std']

In [ ]:
# cols_orig_to_drop = [
#     cols_to_drop_generator, cols_to_drop_hyd_group, cols_to_drop_gen_bear,
#     cols_to_drop_transf, cols_to_drop_gearbox
# ]

In [ ]:
# for i, key in enumerate(comp_prep_df_dict):
#     comp_prep_df_dict[key]= comp_prep_df_dict[key].drop(columns=cols_orig_to_drop[i])

In [ ]:
for key in comp_prep_df_dict:
    print(comp_prep_df_dict[key].shape)

# Train and test split

In [ ]:
# Separar os dataframes de treino e teste
df_train_dict = comp_prep_df_dict.copy()
df_test_dict = comp_prep_df_dict.copy()

per_test = '3meses'

for key in df_train_dict:
    df_train_dict[key] = dprep.prepare_train_df(df_train_dict[key], meses=3)
    df_test_dict[key] = dprep.prepare_test_df(df_test_dict[key], meses=3)

In [ ]:
train_turbines = {
    'df_generator': ['T11', 'T06'],
    'df_hydraulic': ['T06', 'T11'],
    'df_gen_bear': ['T07', 'T09'],
    'df_transformer': ['T07'],
    'df_gearbox': ['T09']
}

In [ ]:
for key in df_train_dict:
    print(f'Considerar apenas as turbinas que tiveram falhas no component {key}')
    df_train_dict[key] = df_train_dict[key][df_train_dict[key]['Turbine_ID'].isin(train_turbines[key])]

In [ ]:
for key in df_train_dict:
    print(f'df_train {key} com a forma {df_train_dict[key].shape}')
    print(f'df_test {key} com a forma {df_test_dict[key].shape}')

# Criar o x_train, x_test, y_train e o y_test 

In [ ]:
x_train = df_train_dict.copy()
x_test = df_test_dict.copy()
y_train = df_train_dict.copy()
y_test = df_test_dict.copy()

dias_var_alvo = days_list[0]

In [ ]:
# retirar as colunas que não devem entrar no x
cols_to_drop_train = [
    'Date', 'TTF',
    str(days_list[0]) + '_days', 'Component', 'Component_av', 'Component_sd'
]
for key in x_train:
    x_train[key] = x_train[key].drop(columns=cols_to_drop_train)
    x_test[key] = x_test[key].drop(columns=cols_to_drop_train)

In [ ]:
for key in x_train:
    print(f'x_train {key} com a forma {x_train[key].shape}')
    print(f'x_test {key} com a forma {x_test[key].shape}')

In [ ]:
# considerar apenas as colunas que devem ser a variável alvo
col_to_mantain_test = [str(days_list[0])+'_days']
for key in y_train:
    y_train[key] = y_train[key][col_to_mantain_test]
    y_test[key] = y_test[key][col_to_mantain_test]

In [ ]:
for key in y_test:
    print(f'y_train {key} com a forma {y_train[key].shape}')
    print(f'y_test {key} com a forma {y_test[key].shape}')

# Model training - variables preparation

In [ ]:
# Normalizar os dados
for key in x_train:
    x_train[key], x_test[key] = dprep.scale(x_train[key], x_test[key])

#Sem normalizar os dados
# X_train = x_train[0].drop(columns=['Turbine_ID']).to_numpy()
# X_test = x_test[0].drop(columns=['Turbine_ID']).to_numpy()

In [ ]:
for key in y_train:
    y_train[key] = y_train[key].to_numpy().astype(int)[:,0]
    y_test[key] = y_test[key].to_numpy().astype(int)[:,0]

# Model Training - Logistic Regression

In [ ]:
# Correr o mesmo modelo para todos os componentes
for key in x_train:
    print(f'Trainning component {key}')
    # Train the model
    model_name = 'Logistic Regression'
    clf_LogReg = LogisticRegression(random_state=42, max_iter=1000)
    gs_params = {
        'C': [.01, 0.1, 1.0, 10],
        'solver': ['liblinear', 'lbfgs'],
        'class_weight': [None, 'balanced']
    }
    gs_score = 'roc_auc'

    clf_LogReg, pred_LogReg = dprep.bin_classify(model_name,
                                                 clf_LogReg,
                                                 x_train[key],
                                                 x_test[key],
                                                 y_train[key],
                                                 y_test[key],
                                                 params=gs_params,
                                                 score=gs_score)
    print('\nBest Parameters:\n', clf_LogReg)
    # Obtain metrics
    metrics_dict_miguel = dprep.metrics(y_test[key], pred_LogReg.y_pred,
                                        pred_LogReg.y_score)
    conf_matrix = dprep.conf_matrix(y_test[key], pred_LogReg.y_pred)
    dprep.roc_curve_plot(y_test[key], pred_LogReg.y_score)

    print(f'Calculating Savings to component {key}')
    # Calculate savings
    sav, cf_numb, met_df = dprep.metrics_create_df(df_test_dict[key],
                                                   y_test[key],
                                                   pred_LogReg.y_pred.values,
                                                   key, 15)

    # Store metrics to compare
    metrics_dict['try_' + str(tentativa)] = {
        'Componente': key,
        'Nome_modelo': model_name,
        'dias_var_alvo': col_to_mantain_test,
        'feature_selection': 'Guilherme',
        'Agrup_por': Agrup_por,
        'add_features': 'smooth only',
        'per_test': per_test,
        'model': model_name,
        'Metrics': metrics_dict_miguel,
        'cf_matriX_log_reg': conf_matrix,
        'Savings': sav,
        'cf_bussines_numbers': cf_numb
    }
    tentativa = tentativa + 1

In [ ]:
Savings = 0
for i in metrics_dict:
    if metrics_dict[i]['Nome_modelo'] == 'Logistic Regression':
        Savings = Savings + metrics_dict[i]['Savings']
        print(metrics_dict[i]['Componente'], metrics_dict[i]['Savings'])
Savings        

# Model Training - Random Forrest Classifier

In [ ]:
# Correr o mesmo modelo para todos os componentes
for key in x_train:
    print(f'Trainning component {key}')
    # Train the model
    model_name = 'Random_Forest_Classifier'
    clf_rfc = RandomForestClassifier(random_state=42)
    gs_params = {
        'n_estimators': [300, 500, 750, 800, 900, 1000],
        'criterion': ['gini', 'entropy'],
        'class_weight': ['balanced', None]
    }
    gs_score = 'recall'

    clf_rcf, pred_rcf = dprep.bin_classify(model_name,
                                           clf_rfc,
                                           x_train[key],
                                           x_test[key],
                                           y_train[key],
                                           y_test[key],
                                           params=gs_params,
                                           score=gs_score)
    print('\nBest Parameters:\n', clf_LogReg)
    # Obtain metrics
    metrics_dict_miguel = dprep.metrics(y_test[key], pred_rcf.y_pred,
                                        pred_rcf.y_score)
    conf_matrix = dprep.conf_matrix(y_test[key], pred_rcf.y_pred)
    dprep.roc_curve_plot(y_test[key], pred_rcf.y_score)

    print(f'Calculating Savings to component {key}')
    # Calculate savings
    sav, cf_numb, met_df = dprep.metrics_create_df(df_test_dict[key],
                                                   y_test[key],
                                                   pred_rcf.y_pred.values,
                                                   key, 15)

    # Store metrics to compare
    metrics_dict['try_' + str(tentativa)] = {
        'Componente': key,
        'Nome_modelo': model_name,
        'dias_var_alvo': col_to_mantain_test,
        'feature_selection': 'Guilherme',
        'Agrup_por': Agrup_por,
        'add_features': 'smooth only',
        'per_test': per_test,
        'model': model_name,
        'Metrics': metrics_dict_miguel,
        'cf_matriX_log_reg': conf_matrix,
        'Savings': sav,
        'cf_bussines_numbers': cf_numb
    }
    tentativa = tentativa + 1

In [ ]:
Savings = 0
for i in metrics_dict:
    if metrics_dict[i]['Nome_modelo'] == 'Random_Forest_Classifier':
        Savings = Savings + metrics_dict[i]['Savings']
        print(metrics_dict[i]['Componente'], metrics_dict[i]['Savings'])
Savings        

# Model Training - Gradient Boost Classifier

In [ ]:
# Correr o mesmo modelo para todos os componentes
for key in x_train:
    print(f'Trainning component {key}')
    # Train the model
    model_name = 'Gradient_Boosting_Classifier'
    clf_gbc = GradientBoostingClassifier(random_state=42)
    gs_params = {
        'learning_rate': [0.001, 0.01, 0.1, 0.5, 1],
        'n_estimators': [100, 200, 500, 700]
    }
    gs_score = 'recall'

    clf_gbc, pred_gbc = dprep.bin_classify(model_name,
                                           clf_gbc,
                                           x_train[key],
                                           x_test[key],
                                           y_train[key],
                                           y_test[key],
                                           params=gs_params,
                                           score=gs_score)
    print('\nBest Parameters:\n', clf_LogReg)
    # Obtain metrics
    metrics_dict_miguel = dprep.metrics(y_test[key], pred_gbc.y_pred,
                                        pred_gbc.y_score)
    conf_matrix = dprep.conf_matrix(y_test[key], pred_gbc.y_pred)
    dprep.roc_curve_plot(y_test[key], pred_gbc.y_score)

    print(f'Calculating Savings to component {key}')
    # Calculate savings
    sav, cf_numb, met_df = dprep.metrics_create_df(df_test_dict[key],
                                                   y_test[key],
                                                   pred_gbc.y_pred.values,
                                                   key, 15)

    # Store metrics to compare
    metrics_dict['try_' + str(tentativa)] = {
        'Componente': key,
        'Nome_modelo': model_name,
        'dias_var_alvo': col_to_mantain_test,
        'feature_selection': 'Guilherme',
        'Agrup_por': Agrup_por,
        'add_features': 'smooth only',
        'per_test': per_test,
        'model': model_name,
        'Metrics': metrics_dict_miguel,
        'cf_matriX_log_reg': conf_matrix,
        'Savings': sav,
        'cf_bussines_numbers': cf_numb
    }
    tentativa = tentativa + 1

In [ ]:
Savings = 0
for i in metrics_dict:
    if metrics_dict[i]['Nome_modelo'] == 'Gradient_Boosting_Classifier':
        Savings = Savings + metrics_dict[i]['Savings']
        print(metrics_dict[i]['Componente'], metrics_dict[i]['Savings'])
Savings        

# Model Training - KNN Classsifier

In [ ]:
# Correr o mesmo modelo para todos os componentes
for key in x_train:
    print(f'Trainning component {key}')
    # Train the model
    model_name = 'KNN_Classifier'
    clf_knn = KNeighborsClassifier()
    gs_params = {
        'n_neighbors': [5, 7, 10, 15],
        'weights': ['uniform', 'distance']
    }
    gs_score = 'recall'

    clf_knn, pred_knn = dprep.bin_classify(model_name,
                                           clf_knn,
                                           x_train[key],
                                           x_test[key],
                                           y_train[key],
                                           y_test[key],
                                           params=gs_params,
                                           score=gs_score)
    print('\nBest Parameters:\n', clf_LogReg)
    # Obtain metrics
    metrics_dict_miguel = dprep.metrics(y_test[key], pred_knn.y_pred,
                                        pred_knn.y_score)
    conf_matrix = dprep.conf_matrix(y_test[key], pred_knn.y_pred)
    dprep.roc_curve_plot(y_test[key], pred_knn.y_score)

    print(f'Calculating Savings to component {key}')
    # Calculate savings
    sav, cf_numb, met_df = dprep.metrics_create_df(df_test_dict[key],
                                                   y_test[key],
                                                   pred_knn.y_pred.values,
                                                   key, 15)

    # Store metrics to compare
    metrics_dict['try_' + str(tentativa)] = {
        'Componente': key,
        'Nome_modelo': model_name,
        'dias_var_alvo': col_to_mantain_test,
        'feature_selection': 'Guilherme',
        'Agrup_por': Agrup_por,
        'add_features': 'smooth only',
        'per_test': per_test,
        'model': model_name,
        'Metrics': metrics_dict_miguel,
        'cf_matriX_log_reg': conf_matrix,
        'Savings': sav,
        'cf_bussines_numbers': cf_numb
    }
    tentativa = tentativa + 1

In [ ]:
Savings = 0
for i in metrics_dict:
    if metrics_dict[i]['Nome_modelo'] == 'KNN_Classifier':
        Savings = Savings + metrics_dict[i]['Savings']
        print(metrics_dict[i]['Componente'], metrics_dict[i]['Savings'])
Savings        

# Model Training - AdaBoostClassifier

In [ ]:
# Correr o mesmo modelo para todos os componentes
for key in x_train:
    print(f'Trainning component {key}')
    # Train the model
    model_name = 'AdaBoostClassifier'
    clf_abc = AdaBoostClassifier(random_state=42)
    gs_params = {'n_estimators': [50, 100, 200, 300, 500]}
    gs_score = 'recall'

    clf_abc, pred_abc = dprep.bin_classify(model_name,
                                           clf_abc,
                                           x_train[key],
                                           x_test[key],
                                           y_train[key],
                                           y_test[key],
                                           params=gs_params,
                                           score=gs_score)
    print('\nBest Parameters:\n', clf_LogReg)
    # Obtain metrics
    metrics_dict_miguel = dprep.metrics(y_test[key], pred_abc.y_pred,
                                        pred_abc.y_score)
    conf_matrix = dprep.conf_matrix(y_test[key], pred_abc.y_pred)
    dprep.roc_curve_plot(y_test[key], pred_abc.y_score)

    print(f'Calculating Savings to component {key}')
    # Calculate savings
    sav, cf_numb, met_df = dprep.metrics_create_df(df_test_dict[key],
                                                   y_test[key],
                                                   pred_abc.y_pred.values,
                                                   key, 15)

    # Store metrics to compare
    metrics_dict['try_' + str(tentativa)] = {
        'Componente': key,
        'Nome_modelo': model_name,
        'dias_var_alvo': col_to_mantain_test,
        'feature_selection': 'Guilherme',
        'Agrup_por': Agrup_por,
        'add_features': 'smooth only',
        'per_test': per_test,
        'model': model_name,
        'Metrics': metrics_dict_miguel,
        'cf_matriX_log_reg': conf_matrix,
        'Savings': sav,
        'cf_bussines_numbers': cf_numb
    }
    tentativa = tentativa + 1

In [ ]:
Savings = 0
for i in metrics_dict:
    if metrics_dict[i]['Nome_modelo'] == 'AdaBoostClassifier':
        Savings = Savings + metrics_dict[i]['Savings']
        print(metrics_dict[i]['Componente'], metrics_dict[i]['Savings'])
Savings        

In [ ]:
import csv

w = csv.writer(open("output5.csv", "w"))
for key, val in metrics_dict.items():
    w.writerow([key, val])

# Cria de DF para avaliação de métricas

In [ ]:
sav

In [ ]:
cf_numb

In [ ]:
met_df[met_df.FP==1][met_df[met_df.FP==1].new_FP==1]